In [210]:
# !pip install ydata_profiling
#Import all relevant libraries
import numpy as np
import pandas as pd
import plotly.express as px
from ydata_profiling import ProfileReport
from scipy import stats
import re
from datetime import datetime
from scipy.stats import zscore
import operator

 Loading the data
We can now load the dataset into pandas using the read_csv() function. This converts the CSV file into a Pandas dataframe.

In [211]:
# Mount Google Drive to access files in Colab environment
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [212]:
#Read in the csv file and convert to a Pandas dataframe
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/bootcamb/week3/project_W/all_fines.csv')

In [213]:
# Display concise summary of the DataFrame:
# - index range
# - column names, non-null counts, and data types
# - memory usage
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 11 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   id               510 non-null    int64 
 1   picture          510 non-null    object
 2   country          510 non-null    object
 3   price            510 non-null    int64 
 4   authority        510 non-null    object
 5   date             220 non-null    object
 6   org_fined        510 non-null    object
 7   articleViolated  510 non-null    object
 8   type             509 non-null    object
 9   source           272 non-null    object
 10  summary          510 non-null    object
dtypes: int64(2), object(9)
memory usage: 44.0+ KB


In [214]:
# Display the first 5 rows of the DataFrame
df.head()


,id,picture,country,price,authority,date,org_fined,articleViolated,type,source,summary
0,1,https://www.privacyaffairs.com/wp-content/uplo...,Poland,9380,Polish National Personal Data Protection Offic...,NaN,Polish Mayor,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,https://uodo.gov.pl/decyzje/ZSPU.421.3.2019,<p>No data processing agreement has been concl...
1,2,https://www.privacyaffairs.com/wp-content/uplo...,Romania,2500,Romanian National Supervisory Authority for Pe...,NaN,UTTIS INDUSTRIES,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,https://www.dataprotection.ro/?page=A_patra_am...,<p>A controller was sanctioned because he had ...
2,3,https://www.privacyaffairs.com/wp-content/uplo...,Spain,60000,Spanish Data Protection Authority (AEPD),NaN,Xfera Moviles S.A.,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,https://www.aepd.es/resoluciones/PS-00262-2019...,<p>The company had unlawfully processed the pe...
3,4,https://www.privacyaffairs.com/wp-content/uplo...,Spain,8000,Spanish Data Protection Authority (AEPD),NaN,Iberdrola Clientes,Art. 31 GDPR,Failure to cooperate with supervisory authority,https://www.aepd.es/resoluciones/PS-00304-2019...,<p>Iberdrola Clientes violated Article 13 of t...
4,5,https://www.privacyaffairs.com/wp-content/uplo...,Romania,150000,Romanian National Supervisory Authority for Pe...,2019-09-10,Raiffeisen Bank SA,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.dataprotection.ro/?page=Comunicat_...,<p>Raiffeisen Bank Romania did not observe the...


In [215]:
# Generate descriptive statistics for numeric columns in the DataFrame
df.describe()


,id,price
count,510.000000,5.100000e+02
mean,255.500000,5.589662e+05
std,147.368586,3.519232e+06
min,1.000000,0.000000e+00
25%,128.250000,3.000000e+03
50%,255.500000,1.000000e+04
75%,382.750000,5.000000e+04
max,510.000000,5.000000e+07


### Viewing the dataframe
We can get a quick sense of the size of our dataset by using the shape method. This returns a tuple with the number of rows and columns in the dataset.

**Note:**
- Your gun/clock time is the time on the official race clock from the moment the race started (when the gun went off) to when you crossed over the finish line.
- Your net/chip time is the time from when you personally cross the starting line to when you cross the finish.

## 1. Data Profiling:
Data profiling is a comprehensive process of examining the data available in an existing dataset and collecting statistics and information about that data.

In [216]:
# view of the data frame types
df.dtypes

,0
id,int64
picture,object
country,object
price,int64
authority,object
date,object
org_fined,object
articleViolated,object
type,object
source,object


In [217]:
# Create a Pandas Profiling report with a custom title
profile = ProfileReport(df, title="GDPR Fines Data Report")
profile

Output hidden; open in https://colab.research.google.com to view.

The process of profiling differs slightly for categorical and numerical variables due to their inherent differences.

**The two main types of data are:**
- Quantitative (numerical) data
- Qualitative (categorical) data

In [218]:
# check the duplicates values
df.duplicated().sum()

np.int64(0)

### Data Quality Checks
Data quality checks involve the process of ensuring that the data is accurate, complete, consistent, relevant, and reliable.


**Here are typical steps involved in checking data quality:**

#### 1. Reliability:
Evaluate the data's source and collection process to determine its trustworthiness.

In [219]:
# Print source reference
kaggle_url = "https://www.kaggle.com/datasets/andreibuliga1/gdpr-fines-20182020-updated-23012021"
print("Kaggle source:", kaggle_url)


Kaggle source: https://www.kaggle.com/datasets/andreibuliga1/gdpr-fines-20182020-updated-23012021


#### 2. Timeliness:
Ensure the fines fall within the expected GDPR-enforcement window:
25-May-2018 → 23-Jan-2021

In [220]:
# Convert date column once (if not yet converted)
df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)

# Basic stats
print("Earliest record :", df["date"].min().date())
print("Latest   record :", df["date"].max().date())

Earliest record : 1970-01-01
Latest   record : 2020-12-11


#### 3. Consistency:

Check duplicates, impossible values, and cross-field mismatches.


In [221]:
# --- A) Duplicate check on key fields ------------
key = ["org_fined", "country", "price", "date"]
dup_total = df.duplicated(subset=key).sum()
print(f"Duplicate rows on {key}: {dup_total}")

# Preview first few duplicates
if dup_total:
    display(df[df.duplicated(subset=key, keep=False)].head())

# --- B) Non-positive or missing fine amounts -----
bad_price = df[df["price"] <= 0]
print("Rows with non-positive fines:", bad_price.shape[0])

# --- C) Country vs Authority quick sanity -------
mask_mismatch = df["authority"].str.contains("CNIL", na=False) & (df["country"] != "FR")
print("Authority–country mismatches:", mask_mismatch.sum())

# --- D) Numerical ranges sanity ------------------
print("Fine € 5th–95th percentiles:",
      df["price"].quantile([0.05, 0.95]).values)

print("Consistency checks finished.")

Duplicate rows on ['org_fined', 'country', 'price', 'date']: 16


,id,picture,country,price,authority,date,org_fined,articleViolated,type,source,summary
2,3,https://www.privacyaffairs.com/wp-content/uplo...,Spain,60000,Spanish Data Protection Authority (AEPD),NaT,Xfera Moviles S.A.,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,https://www.aepd.es/resoluciones/PS-00262-2019...,<p>The company had unlawfully processed the pe...
6,7,https://www.privacyaffairs.com/wp-content/uplo...,Greece,200000,Hellenic Data Protection Authority (HDPA),2019-10-07,Telecommunication Service Provider,"Art. 5 (1) c) GDPR, Art. 25 GDPR",Failure to comply with data processing principles,https://www.dpa.gr/APDPXPortlets/htdocs/docume...,<p>Despite the clear refusal of telemarketing ...
7,8,https://www.privacyaffairs.com/wp-content/uplo...,Greece,200000,Hellenic Data Protection Authority (HDPA),2019-10-07,Telecommunication Service Provider,"Art. 21 (3) GDPR, Art. 25 GDPR",Failure to comply with data processing principles,https://www.dpa.gr/APDPXPortlets/htdocs/docume...,"<p>Due to technical errors, the personal data ..."
79,80,https://www.privacyaffairs.com/wp-content/uplo...,Germany,80000,Data Protection Authority of Baden-Wuerttemberg,NaT,Unknown,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.baden-wuerttemberg.datenschutz.de/...,<p>Two companies working in finances didn’t fo...
80,81,https://www.privacyaffairs.com/wp-content/uplo...,Germany,80000,Data Protection Authority of Baden-Wuerttemberg,NaT,Unknown,Art. 32 GDPR,Failure to implement sufficient measures to en...,https://www.baden-wuerttemberg.datenschutz.de/...,<p>Because of insufficient data security mecha...


Rows with non-positive fines: 9
Authority–country mismatches: 10
Fine € 5th–95th percentiles: [5.0000e+02 7.6625e+05]
Consistency checks finished.


#### 4. Relevance:
Assess whether the data is appropriate and applicable for the intended analysis. Data that is not relevant can skew results and lead to incorrect conclusions.

**Key considerations for relevance include:**

> 1. Sample Appropriateness: Confirm that your data sample aligns with your analysis objectives. For instance, utilizing data from the Northern region will not yield accurate insights for the Western region of the Kingdom.
>
> 2. Variable Selection: Any column will not be relevant for our analysis, we can get rid of these using the drop() method. We will set the “axis” argument to 1 since we’re dealing with columns, and set the “inplace” argument to True to make the change permanent.


In [222]:
# Keep only the specified columns and create a copy of the DataFrame
cols_to_keep = ["date", "country", "org_fined",
                "price", "articleViolated",
                "type", "summary"]
df = df[cols_to_keep].copy()
# Print the list of remaining column names
print("Remaining columns:", list(df.columns))

Remaining columns: ['date', 'country', 'org_fined', 'price', 'articleViolated', 'type', 'summary']


In [223]:
# Display concise summary of the DataFrame:
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             220 non-null    datetime64[ns]
 1   country          510 non-null    object        
 2   org_fined        510 non-null    object        
 3   price            510 non-null    int64         
 4   articleViolated  510 non-null    object        
 5   type             509 non-null    object        
 6   summary          510 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 28.0+ KB


In [224]:
df.head() # Display the first 5 rows of the new DataFrame


,date,country,org_fined,price,articleViolated,type,summary
0,NaT,Poland,Polish Mayor,9380,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,<p>No data processing agreement has been concl...
1,NaT,Romania,UTTIS INDUSTRIES,2500,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,<p>A controller was sanctioned because he had ...
2,NaT,Spain,Xfera Moviles S.A.,60000,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,<p>The company had unlawfully processed the pe...
3,NaT,Spain,Iberdrola Clientes,8000,Art. 31 GDPR,Failure to cooperate with supervisory authority,<p>Iberdrola Clientes violated Article 13 of t...
4,2019-10-09,Romania,Raiffeisen Bank SA,150000,Art. 32 GDPR,Failure to implement sufficient measures to en...,<p>Raiffeisen Bank Romania did not observe the...


#### 5. Uniqueness:
Check for and remove duplicate records to prevent skewed analysis results.


In [225]:
df.duplicated().sum() # Count how many rows are exact duplicates across all columns

np.int64(0)

In [226]:
# Get the dimensions of the DataFrame as (rows, columns)
df.shape

(510, 7)

#### 6. Completeness:
Ensure that no critical data is missing. This might mean checking for null values or required fields that are empty.

We will start by checking the dataset for missing or null values. For this, we can use the isna() method which returns a dataframe of boolean values indicating if a field is null or not. To group all missing values by column, we can include the sum() method.

In [227]:
# Count missing (NaN) values for each column in the DataFrame
df.isna().sum()


,0
date,290
country,0
org_fined,0
price,0
articleViolated,0
type,1
summary,0


"""
Completeness handling:
1. Try to infer / fill missing `date`.
2. Fill missing `type` with 'Unknown'.
3. Create two DataFrames:
   • df_fulltime  -> records WITH date (for time series charts)
   • df_model     -> all records (date not mandatory for ML models)
"""

In [228]:
# A) keep everything for ML & general EDA
df_model = df.copy()

# B) exact-date subset for time-series plots
df_time = df[df["date"].notna()].copy()

print("Rows for modelling :", df_model.shape[0])
print("Rows with valid date:", df_time.shape[0])




Rows for modelling : 510
Rows with valid date: 220


#### 7. Check Accuracy:

Verify that the data is correct and precise. This could involve comparing data samples with known sources or using validation rules.

**The process includes:**
1. Validating the appropriateness of data types for the dataset.
2. Identifying outliers  using established validation  rule

In [229]:
# Preview the first 5 rows to inspect data structure and sample values
df.head()

# Convert 'date' column to datetime; invalid parses become NaT (missing)
df["date"] = pd.to_datetime(df["date"], errors="coerce")

In [230]:
# check columns types
df.dtypes

,0
date,datetime64[ns]
country,object
org_fined,object
price,int64
articleViolated,object
type,object
summary,object


**What is an Outlier?**
Outlier is an row/observation that appears far away and diverges from an overall pattern in a sample.

**What are the types of Outliers?**
1. Univariate: These outliers can be found when we look at distribution of a single variable
2. Multivariate: are outliers in an n-dimensional space. In order to find them, you have to look at distributions in multi-dimensions. example (hight=100, weight=100) for a person

**What causes Outliers?**
Whenever we come across outliers, the ideal way to tackle them is to find out the reason of having these outliers. The method to deal with them would then depend on the reason of their occurrence.

Let’s understand various types of outliers:

1. Data Entry Errors:- Human errors such as errors caused during data collection, recording, or entry can cause outliers in data.
2. Measurement Error: It is the most common source of outliers. This is caused when the measurement instrument used turns out to be faulty.
3. Data Processing Error: Whenever we perform data mining, we extract data from multiple sources. It is possible that some manipulation or extraction errors may lead to outliers in the dataset.
4. Sampling error: For instance, we have to measure the height of athletes. By mistake, we include a few basketball players in the sample. This inclusion is likely to cause outliers in the dataset.
5. Natural Outlier: When an outlier is not artificial (due to error), it is a natural outlier. For instance: In my last assignment with one of the renowned insurance company, I noticed that the performance of top 50 financial advisors was far higher than rest of the population. Surprisingly, it was not due to any error. Hence, whenever we perform any data mining activity with advisors, we used to treat this segment separately.


**What is the impact of Outliers on a dataset?**


![image.png](https://www.analyticsvidhya.com/wp-content/uploads/2015/02/Outlier_31.png)




**How to detect Outliers?**

1. Most commonly used method to detect outliers is visualization (Univariate Graphical Analysis).

We use 3 common visualization methods:
>- Box-plot: A box plot is a method for graphically depicting groups of numerical data through their quartiles. The box extends from the Q1 to Q3 quartile values of the data, with a line at the median (Q2). The whiskers extend from the edges of the box to show the range of the data. Outlier points are those past the end of the whiskers. Box plots show robust measures of location and spread as well as providing information about symmetry and outliers.
>
>  
>![image.png](https://miro.medium.com/v2/resize:fit:698/format:webp/1*VK5iHA2AB28HSZwWwUbNYg.png)
>
>
>- Histogram
>- Scatter Plot: A scatter plot is a mathematical diagram using Cartesian coordinates to display values for two variables for a set of data. The data are displayed as a collection of points, each having the value of one variable determining the position on the horizontal axis and the value of the other variable determining the position on the vertical axis. The points that are far from the population can be termed as an outlier.
>
>  
>![image.png](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*Ov6aH-8yIwNoUxtMFwgx4g.png)
>
>

2. Using statistical method (Univariate Non-Graphical analysis):
>- Any value, which is beyond the range of -1.5 x IQR to 1.5 x IQR

![image.png](https://www.whatissixsigma.net/wp-content/uploads/2015/07/Box-Plot-Diagram-to-identify-Outliers-figure-1.png)

>- Use capping methods. Any value which out of range of 5th and 95th percentile can be considered as outlier
>- Data points, three or more standard deviation away from mean are considered outlier: The Z-score is the signed number of standard deviations by which the value of an observation or data point is above the mean value of what is being observed or measured. While calculating the Z-score we re-scale and center the data and look for data points that are too far from zero. These data points which are way too far from zero will be treated as the outliers. In most of the cases, a threshold of 3 or -3 is used i.e if the Z-score value is greater than or less than 3 or -3 respectively, that data point will be identified as outliers.
> - Outlier detection is merely a special case of the examination of data for influential data points and it also depends on the business understanding


In [231]:
# Scatter plot of fines vs. calendar year with an OLS trendline
fig = px.scatter(
    df,
    x=df["date"].dt.year,   # extract year from 'date' for the x-axis
    y="price",              # fine amount on the y-axis
    title="Fine vs. Year",
    labels={"x": "Year", "price": "Fine (€)"},
    trendline="ols"         # add ordinary least squares trendline
)

# Use logarithmic scale on y-axis to reduce skew from large fines
fig.update_yaxes(type="log")

# Render the figure
fig.show()


In [232]:

# ---------- 0) Filter valid GDPR period ----------
df_valid = df[df["date"].notna() & (df["date"] >= "2018-05-25")].copy()

# ---------- 1) Scatter: full date vs price ----------
fig = px.scatter(
    df_valid,
    x="date", y="price",
    title="Fines Over Time (Full Date)",
    labels={"date": "Date", "price": "Fine (€)"},
    trendline="ols"
)
fig.update_yaxes(type="log")  # log scale to handle skew
fig.show()

# Boxplot: distribution per year
df_valid["year"] = df_valid["date"].dt.year
fig = px.box(
    df_valid,
    x="year", y="price",
    points="all",  # show all points for context
    title="Distribution of Fines by Year",
    labels={"year": "Year", "price": "Fine (€)"}
)
fig.update_yaxes(type="log")
fig.show()




In [233]:
# detect outliers using numerical statistics

In [234]:
import numpy as np
import pandas as pd

# --- 1) IQR method ----------------------------------------------------------
Q1, Q3 = df["price"].quantile([0.25, 0.75])
IQR     = Q3 - Q1
lower   = Q1 - 1.5 * IQR
upper   = Q3 + 1.5 * IQR

out_iqr = df[(df["price"] < lower) | (df["price"] > upper)]
print(f"IQR thresholds : {lower:,.0f} → {upper:,.0f}")
print("Outliers by IQR :", out_iqr.shape[0])

# --- 2) Z-score method (μ ± 3σ) -------------------------------------------
mu  = df["price"].mean()
std = df["price"].std(ddof=0)
z_upper = mu + 3*std
z_lower = mu - 3*std

out_z = df[(df["price"] < z_lower) | (df["price"] > z_upper)]
print(f"Z-score thresholds : {z_lower:,.0f} → {z_upper:,.0f}")
print("Outliers by Z-score :", out_z.shape[0])

# --- 3) 95th percentile capping suggestion ---------------------------------
p95 = df["price"].quantile(0.95)
print(f"95th percentile cap : {p95:,.0f}")

# Preview rows flagged by both rules
common_outliers = df.merge(out_iqr, how="inner")
print("Example outliers:")
display(common_outliers.head())


IQR thresholds : -67,500 → 120,500
Outliers by IQR : 67
Z-score thresholds : -9,988,374 → 11,106,306
Outliers by Z-score : 9
95th percentile cap : 766,250
Example outliers:


,date,country,org_fined,price,articleViolated,type,summary
0,2019-10-09,Romania,Raiffeisen Bank SA,150000,Art. 32 GDPR,Failure to implement sufficient measures to en...,<p>Raiffeisen Bank Romania did not observe the...
1,2019-10-07,Greece,Telecommunication Service Provider,200000,"Art. 5 (1) c) GDPR, Art. 25 GDPR",Failure to comply with data processing principles,<p>Despite the clear refusal of telemarketing ...
2,2019-10-07,Greece,Telecommunication Service Provider,200000,"Art. 21 (3) GDPR, Art. 25 GDPR",Failure to comply with data processing principles,"<p>Due to technical errors, the personal data ..."
3,NaT,Germany,Delivery Hero,195407,"Art. 15 GDPR, Art. 17 GDPR, Art. 21 GDPR",Non-compliance with subjects' rights protectio...,<p>The Company had retained the personal data ...
4,2019-09-10,Poland,Morele.net,644780,Art. 32 GDPR,Failure to implement sufficient measures to en...,<p>Morele.net was sanctioned with a fine of PL...


## 2. Data Cleaning:

Preliminary findings from data profiling can lead to cleaning the data by:
- Handling missing values
- Correcting errors.
- Dealing with outliers.

![image.png](https://miro.medium.com/v2/resize:fit:4800/format:webp/1*ILbJXAzcB3hU4LxAMG6yLw.jpeg)

-------------------



### Handling missing values:

In [235]:
# Display concise summary of the DataFrame:
# - index range
# - column names, non-null counts, and data types
# - memory usage
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 510 entries, 0 to 509
Data columns (total 7 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   date             220 non-null    datetime64[ns]
 1   country          510 non-null    object        
 2   org_fined        510 non-null    object        
 3   price            510 non-null    int64         
 4   articleViolated  510 non-null    object        
 5   type             509 non-null    object        
 6   summary          510 non-null    object        
dtypes: datetime64[ns](1), int64(1), object(5)
memory usage: 28.0+ KB


In [236]:
# Count unique values in the 'type' column, sorted by frequency (descending)
df['type'].value_counts()

,count
type,
Non-compliance with lawful basis for data processing,178
Failure to implement sufficient measures to ensure information security,110
Failure to comply with data processing principles,105
Non-compliance with subjects' rights protection safeguards,37
Information obligation non-compliance,25
Non-cooperation with Data Protection Authority,12
Failure to comply with processing principles,10
Failure to comply with Data Processing Authority's orders,4
Non-compliance with the right of consent,3


In [237]:
# Remove rows where 'type' column has missing (NaN) values
df = df.dropna(subset=['type'])

In [238]:
# Count number of missing (NaN) values in the 'type' column
df['type'].isna().sum()


np.int64(0)

In [239]:
# Create a flag column: 1 if 'date' is missing (NaN), else 0
df['date_filled'] = df['date'].isna().astype(int)


/tmp/ipython-input-1413575366.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [240]:
default_date = pd.Timestamp('2020-06-15')
df['date'] = df['date'].fillna(default_date)

/tmp/ipython-input-1543977254.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [241]:
df.head()

,date,country,org_fined,price,articleViolated,type,summary,date_filled
0,2020-06-15,Poland,Polish Mayor,9380,Art. 28 GDPR,Non-compliance with lawful basis for data proc...,<p>No data processing agreement has been concl...,1
1,2020-06-15,Romania,UTTIS INDUSTRIES,2500,"Art. 12 GDPR, Art. 13 GDPR, Art. 5 (1) c) GDPR...",Information obligation non-compliance,<p>A controller was sanctioned because he had ...,1
2,2020-06-15,Spain,Xfera Moviles S.A.,60000,"Art. 5 GDPR, Art. 6 GDPR",Non-compliance with lawful basis for data proc...,<p>The company had unlawfully processed the pe...,1
3,2020-06-15,Spain,Iberdrola Clientes,8000,Art. 31 GDPR,Failure to cooperate with supervisory authority,<p>Iberdrola Clientes violated Article 13 of t...,1
4,2019-10-09,Romania,Raiffeisen Bank SA,150000,Art. 32 GDPR,Failure to implement sufficient measures to en...,<p>Raiffeisen Bank Romania did not observe the...,0


### Dealing with outliers:

**How to remove Outliers?**
Most of the ways to deal with outliers are similar to the methods of missing values like deleting rows, transforming them, binning them, treat them as a separate group, imputing values and other statistical methods. Here, we will discuss the common techniques used to deal with outliers:

1. Deleting rows: We delete outlier values if it is due to data entry error, data processing error or outlier rows are very small in numbers. We can also use trimming at both ends to remove outliers.

2. Imputing: Like imputation of missing values, we can also impute outliers. We can use mean, median, mode imputation methods. Before imputing values, we should analyse if it is natural outlier or artificial. If it is artificial, we can go with imputing values. We can also use statistical model to predict values of outlier rows and after that we can impute it with predicted values.

3. Treat separately: If there are significant number of outliers, we should treat them separately in the statistical model. One of the approach is to treat both groups as two different groups and build individual model for both groups and then combine the output.


In [242]:
# =========================
# Outliers Detection
# =========================

num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
print("Numeric columns:", num_cols)


# ---------- (Percentile caps (Winsor bounds only) ----------
def percentile_bounds(s: pd.Series, low=0.01, high=0.99):
    return s.quantile(low), s.quantile(high)

def detect_outliers_percentile(df: pd.DataFrame, cols, low=0.01, high=0.99):
    summary, masks = [], {}
    for c in cols:
        lb, ub = percentile_bounds(df[c].dropna(), low, high)
        mask = (df[c] < lb) | (df[c] > ub)
        masks[c] = mask
        examples = df.loc[mask, c].head(5).tolist()
        pct = mask.mean()*100
        summary.append((c, int(mask.sum()), lb, ub, pct, examples))
    summary_p = pd.DataFrame(summary, columns=["column","outlier_count","lower_bound","upper_bound","pct","examples"])
    any_mask = pd.DataFrame(masks).any(axis=1)
    return summary_p.sort_values("outlier_count", ascending=False), any_mask, masks


summary_p, any_p, masks_p = detect_outliers_percentile(df, num_cols, low=0.01, high=0.99)



df['is_outlier_p_any']   = any_p.astype(int)

print("Rows with any P outlier  :", df['is_outlier_p_any'].sum())


Numeric columns: ['price', 'date_filled']
Rows with any P outlier  : 6


In [243]:
# Count rows flagged as percentile outliers
removed = int(any_p.sum())

# Drop rows where any_p == True and reset index
df = df.loc[~any_p].reset_index(drop=True)

# Report how many rows were removed and the new shape
print(f"Removed {removed} outlier rows. New shape: {df.shape}")


Removed 6 outlier rows. New shape: (503, 9)


### Feature engineering:





In [244]:
# Parse 'date' to datetime; invalid formats become NaT (missing)
df["date"] = pd.to_datetime(df["date"], errors="coerce", dayfirst=True)

# Sort rows chronologically by date
df = df.sort_values("date")

# Extract calendar year from the date
df["year"] = df["date"].dt.year

# Derive year-month as a string (YYYY-MM)
df["month"] = df["date"].dt.to_period("M").astype(str)

# Compute log-transformed price with +1 to handle zeros safely
df["log_price"] = np.log1p(df["price"])


In [245]:
# Count number of violations per article (Art. 1 .. Art. 99)

Art_dict = {}                         # holds {article_number: count}
art_num = np.arange(1, 100)           # candidate article numbers 1..99

for num in art_num:
    for index, row in df.iterrows():  # iterate rows of the DataFrame
        # check if the text contains pattern like "Art. {num} "
        if operator.contains(row['articleViolated'], f'Art. {num} '):
            if num in Art_dict:
                Art_dict[num] = Art_dict[num] + 1  # increment count
            else:
                Art_dict[num] = 1                  # first occurrence

# build a DataFrame from the dictionary and sort by count
Articales = pd.DataFrame(Art_dict.items(), columns=['Articale', 'Violations']).sort_values('Violations')

# ensure article numbers are strings for plotting/labeling
Articales['Articale'] = Articales['Articale'].astype(str)

# preview
Articales.head()


,Articale,Violations
3,8,1
9,16,1
13,23,1
17,29,1
26,48,1


## 3. Univariate Analysis:

This involves examining single variables to understand their characteristics (distribution, central tendency, dispersion, and shape).

We calculate **numerical values** about the data that tells us about the distribution of the data. We also **draw graphs** showing visually how the data is distributed. **To answer the following questions about Features/characteristics of Data:**
- Where is the center of the data? (location)
- How much does the data vary? (scale)
- What is the shape of the data? (shape)

**The benefits of this analysis:**
Statistics summary gives a high-level idea to identify whether the data has any outliers, data entry error, distribution of data such as the data is normally distributed or left/right skewed

**In this step, we will explore variables one by one using following approaches:**

### 1. Univariate Graphical Analysis:
Method to perform uni-variate analysis will depend on whether the variable type is categorical or numerical.

In [246]:
df.head(2)

,date,country,org_fined,price,articleViolated,type,summary,date_filled,is_outlier_p_any,year,month,log_price
88,1970-01-01,Spain,ENDESA,60000,Art. 5 (1) f) GDPR,Non-compliance with lawful basis for data proc...,<p>ENDESA erroneously charged the claimant’s b...,0,0,1970,1970-01,11.002117
95,1970-01-01,Germany,Unknown,500,Art. 6 GDPR,Non-compliance with lawful basis for data proc...,<p>Not available.</p>\n,0,0,1970,1970-01,6.216606


In [247]:
df.dtypes # Display data types of all columns in the DataFrame

,0
date,datetime64[ns]
country,object
org_fined,object
price,int64
articleViolated,object
type,object
summary,object
date_filled,int64
is_outlier_p_any,int64
year,int32


#### I. Categorical Variables:

we’ll use frequency table to understand distribution of each category
- Bar Chart (Ordinal) - Orderd
- Pie Chart (Nominal) - non Orderd

In [248]:
# Number of Fines per Country
country_count =df.country.value_counts().reset_index()
country_count.columns =['Country_Name','Number_of_Fines']
country_count.head()


# number of fines per country
fig = px.bar(data_frame = country_count,
             x ='Country_Name',y='Number_of_Fines',
             text = 'Number_of_Fines',
             title ='Number of Fines per Country',
             color= 'Number_of_Fines',
             log_y =True)
fig.update_traces(textposition='outside')
fig.update_xaxes(title_text ='Country')
fig.update_yaxes(title_text ='Num. of Fines')
fig.show()



In [249]:
# Bar: Top-10 countries by total fine (colored by value)
top_country_fine = (
    df.groupby("country")["price"]
      .sum()
      .nlargest(10)
      .reset_index()
      .rename(columns={"country":"Country", "price":"TotalFine"})
)

order = top_country_fine.sort_values("TotalFine", ascending=False)["Country"].tolist()
top_country_fine["Country"] = pd.Categorical(top_country_fine["Country"], categories=order, ordered=True)

fig = px.bar(
    top_country_fine.sort_values("TotalFine", ascending=False),
    x="Country", y="TotalFine",
    color="TotalFine",
    color_continuous_scale=px.colors.sequential.Viridis,
    title="Top-10 Countries – Total Fine (€)",
    labels={"Country":"Country", "TotalFine":"Total (€)"},
    text="TotalFine"
)

fig.update_traces(texttemplate="%{text:,.0f}", textposition="outside", cliponaxis=False)
fig.update_yaxes(tickformat=",")



fig.show()


In [250]:
# Pie: Violation-type share (Top-6 + Other)
v_counts = (
    df["type"]
      .value_counts()
      .reset_index()
)
v_counts.columns = ["violation", "count"]   # explicit rename

# Build Top-6 + "Other"
v_top6 = v_counts.head(6).copy()
other_sum = v_counts["count"].iloc[6:].sum()
v_top6.loc[len(v_top6)] = ["Other", other_sum]

# Pie chart
fig = px.pie(v_top6,
             names="violation",
             values="count",
             title="Violation-Type Share (Top-6)")
fig.show()


In [251]:
# Group by organization and sum total fines, then get top 20
top_orgs = (df.groupby("org_fined")["price"]
              .sum()
              .nlargest(20)
              .reset_index(name="total"))

# Create treemap showing top 20 organizations by total fine
fig = px.treemap(top_orgs, path=["org_fined"], values="total",
                 title="Top-20 Organisations by Total Fine (€)",
                 hover_data={"total":":,.0f"})

# Display the chart
fig.show()


In [252]:
# Top-5 countries by total count
top5_countries = df["country"].value_counts().head(5).index
subset = df[df["country"].isin(top5_countries)].copy()

# --- Order countries by total descending ---
country_order = (subset.groupby("country")
                        .size()
                        .sort_values(ascending=False)
                        .index.tolist())

# Group by country & violation type
stacked = (subset
           .groupby(["country", "type"], as_index=False)
           .size()
           .rename(columns={"size": "count"}))

# Make 'country' categorical to enforce order
stacked["country"] = pd.Categorical(stacked["country"],
                                    categories=country_order,
                                    ordered=True)

# COUNT (stacked)
fig = px.bar(
    stacked.sort_values(["country","count"], ascending=[True, False]),
    x="country", y="count", color="type",
    title="Violation-type Distribution within Top-5 Countries (Counts)",
    labels={"count": "Count", "type": "Violation Type"},
    text="count",
    category_orders={"country": country_order}
)
fig.update_layout(barmode="stack", xaxis_title="Country", yaxis_title="Count")
fig.show()




#### II. Numerical Variables:

we need to understand the central tendency and spread of the variable (Descriptive Analysis) using:
   - Box plot
   - Histogram

In [253]:
# Number of Violations per Article
fig = px.bar(data_frame = Articales,
             x ='Articale',y='Violations',
             text = 'Violations',
             title ='Number of Violations per Article',
             color= 'Violations',
             log_y =True)
fig.update_traces(textposition='outside')
fig.update_xaxes(title_text ='GDPR Articale num')
fig.update_yaxes(title_text ='Num. of Violations')
fig.show()

In [254]:
#  Histogram – log-fines
fig = px.histogram(df, x=np.log1p(df["price"]), nbins=30,
                   title="Histogram of log(Fine)",
                   labels={"x": "log(Fine+1)"})
fig.show()

In [255]:
#  ECDF (Empirical CDF)
fig = px.ecdf(df, x="price",
              title="ECDF of Fines",
              labels={"price": "Fine (€)"})
fig.update_xaxes(type="log")
fig.show()

In [256]:
# Filter rows with valid dates on/after GDPR start (2018-05-25)
time_df = df[df["date"].notna() & (df["date"] >= "2018-05-25")].copy()

# Sort by date and keep only needed columns
cum = time_df.sort_values("date")[["date", "price"]]

# Compute running (cumulative) total of fines
cum["cum_total"] = cum["price"].cumsum()

# Plot cumulative fines over time
fig = px.line(cum, x="date", y="cum_total",
              title="Cumulative GDPR Fines (2018 →)",
              labels={"cum_total": "Cumulative Total (€)", "date": "Date"})

# Show the chart
fig.show()


In [257]:
# Derive month start for each record (normalize dates to month)
time_df["month_start"] = time_df["date"].dt.to_period("M").dt.to_timestamp()

# Aggregate number of fines per month
monthly_cnt = (time_df
               .groupby("month_start")
               .size()
               .reset_index(name="count"))

# Plot monthly fine counts with markers
fig = px.line(monthly_cnt, x="month_start", y="count", markers=True,
              title="Monthly Fine Count",
              labels={"month_start": "Month", "count": "Number of Fines"})

# Format x-axis as YYYY-MM and tilt labels
fig.update_xaxes(tickformat="%Y-%m", tickangle=-45)

# Show the chart
fig.show()


In [258]:
# Keep rows with valid dates on/after GDPR start and extract year
valid_time = df[df["date"].notna() & (df["date"] >= "2018-05-25")].copy()
valid_time["year"] = valid_time["date"].dt.year

# Get top 3 countries by record count
top3 = valid_time["country"].value_counts().head(3).index

# Aggregate yearly total fines for the top 3 countries
area_df = (valid_time[valid_time["country"].isin(top3)]
           .groupby(["year", "country"])["price"]
           .sum()
           .reset_index())

# Stacked area chart of yearly totals by country
fig = px.area(area_df, x="year", y="price", color="country",
              title="Yearly Total Fine – Top-3 Countries (2018 →)",
              labels={"price": "Total (€)", "year": "Year"})

# Show chart
fig.show()


In [259]:
#  Monthly total fines (line)
monthly = (df_valid
           .groupby(df_valid["date"].dt.to_period("M"))["price"]
           .sum()
           .reset_index(name="total_price"))
monthly["month"] = monthly["date"].dt.to_timestamp()

fig = px.line(
    monthly,
    x="month", y="total_price",
    title="Monthly Total GDPR Fines",
    labels={"month": "Month", "total_price": "Total (€)"}
)
fig.update_yaxes(type="log")  # optional: comment out if you prefer linear
fig.show()